# WSL Merge

Merge score data with date and time data

## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## Scores Data

In [2]:
df = pd.read_csv("../data/wsl/avg_scores_all_years.csv", index_col=0).rename(columns={'top_two_waves_total': 'score'})
df.head()


,year,event,round,heat,name,score
0,2008,Quiksilver Pro Gold Coast,Round 2,Heat 1,Andy Irons,16.37
1,2008,Quiksilver Pro Gold Coast,Round 2,Heat 1,Tamaroa McComb,5.90
2,2008,Quiksilver Pro Gold Coast,Round 2,Heat 2,Julian Wilson,14.00
3,2008,Quiksilver Pro Gold Coast,Round 2,Heat 2,Pancho Sullivan,13.23
4,2008,Quiksilver Pro Gold Coast,Round 2,Heat 3,Jeremy Flores,10.83


Look at unique events and find total amoun of heats for the Pipeline events. Event names change with sponsorship changes.

In [3]:
df["event"].unique()


array(['Quiksilver Pro Gold Coast', 'Rip Curl Pro Bells Beach',
       'Billabong Pro Teahupoo', 'Globe Pro Fiji', 'Billabong Pro J-Bay',
       'Rip Curl Pro Search Bali', 'Boost Mobile Pro',
       'Quiksilver Pro France', 'Billabong Pro Mundaka',
       'Hang Loose Santa Catarina Pro', 'Billabong Pipeline Masters',
       'Hurley Pro Trestles', 'Rip Curl Search', 'Santa Catarina Pro',
       'Rip Curl Pro Portugal', 'Billabong Pipe Masters',
       'Billabong Rio Pro', 'Quiksilver Pro New York', 'Volcom Fiji Pro',
       "O'Neill Coldwater Classic Santa Cruz", 'Oakley Pro Bali',
       'Drug Aware Margaret River Pro', 'Fiji Pro', 'J-Bay Open',
       'Billabong Pro Tahiti', 'Hurley Pro at Trestles',
       'Moche Rip Curl Pro Portugal', 'Oi Rio Pro',
       'MEO Rip Curl Pro Portugal', 'Outerknown Fiji Pro',
       'Corona Open J-Bay', 'Corona Bali Protected',
       'Uluwatu CT Margaret River Pro', "Tahiti Pro Teahupo'o",
       'Margaret River Pro',
       'Billabong Pipe Masters 

Locate rows with pipeline events and get the shape of the resulting dataframe. This is the amount of heats in pipeline events with data.

In [4]:
# Make list of pipeline events from previous cell
pipe = [
    "Billabong Pipe Masters presented by Hydro Flask",
    "Billabong Pro Pipeline",
    "Billabong Pipe Masters presented by Hydro Flask",
    "Billabong Pipe Masters",
    "Billabong Pipeline Masters",
]

df.loc[df["event"].isin(pipe)].shape


(1649, 6)

Check all other events to confirm pipeline has the most data.

In [5]:
df["event"].value_counts().head(15)


Quiksilver Pro Gold Coast        1473
Quiksilver Pro France            1444
Rip Curl Pro Bells Beach         1422
Billabong Pipe Masters           1176
Billabong Pro Teahupoo            729
Hurley Pro Trestles               614
Oi Rio Pro                        574
Billabong Pro J-Bay               493
Hurley Pro at Trestles            472
Billabong Pro Tahiti              472
Billabong Rio Pro                 472
Rip Curl Pro Portugal             472
Drug Aware Margaret River Pro     471
MEO Rip Curl Pro Portugal         456
J-Bay Open                        354
Name: event, dtype: int64

**Pipeline is the event with most data**

Trestles is another good target with data and historical buoy info.

In [6]:
# Cast year data as string to avoid groupby issues
df["year"] = df["year"].astype(str)


### Check and rename rounds and heats for consistency accross events

I've mapped these manually but str methods would reduce the amount of code

In [7]:
# Get unique round names
df["round"].unique()


array(['Round 2', 'Round 1', 'Round 3', 'Round 4', 'Quarterfinal',
       'Semifinal', 'Final', 'Round 5', 'Quarterfinals', 'Semifinals',
       'Trials Quarterfinals', 'Trials Round 1', 'Trials Semifinals',
       'Trials Final', 'Elimination Round', 'Seeding Round',
       'Round of 32', 'Round of 16', 'Opening Round'], dtype=object)

In [8]:
# Map round names to a consistent standard
# Round 1, 2, 3, 4 and 5, then Quarterfinals, Semifinals and Final. Idem for trials
df["round"] = df["round"].map(
    {
        "Seeding Round": "Round 1",
        "Round of 32": "Round 3",
        "Quarterfinal": "Quarterfinals",
        "Round of 16": "Round 4",
        "Elimination Round": "Round 2",
        "Opening Round": "Round 1",
        "Round 1": "Round 1",
        "Round 2": "Round 2",
        "Round 3": "Round 3",
        "Round 4": "Round 4",
        "Round 5": "Round 5",
        "Quarterfinals": "Quarterfinals",
        "Final": "Final",
        "Semifinal": "Semifinals",
        "Semifinals": "Semifinals",
        "Trials Quarterfinals": "Trials Quarterfinals",
        "Trials Round 1": "Trials Round 1",
        "Trials Semifinals": "Trials Semifinals",
        "Trials Final": "Trials Final",
    }
)


In [9]:
# Get unique heat names
df['heat'].unique()

array(['Heat 1 ', 'Heat 2 ', 'Heat 3 ', 'Heat 4 ', 'Heat 5 ', 'Heat 6 ',
       'Heat 7 ', 'Heat 8 ', 'Heat 9 ', 'Heat 10 ', 'Heat 11 ',
       'Heat 12 ', 'Heat 13 ', 'Heat 14 ', 'Heat 15 ', 'Heat 16 ',
       'Heat 8 Watch replay', 'Heat 6 Watch replay',
       'Heat 2 Watch replay', 'Heat 1 Watch replay',
       'Heat 4 Watch replay', 'Heat 10 Watch replay',
       'Heat 11 Watch replay', 'Heat 12 Watch replay',
       'Heat 3 Watch replay', 'Heat 5 Watch replay',
       'Heat 7 Watch replay', 'Heat 9 Watch replay',
       'Heat 13 Watch replay', 'Heat 14 Watch replay',
       'Heat 15 Watch replay', 'Heat 16 Watch replay'], dtype=object)

In [10]:
# Map for consistency and to eliminate 'Watch replay' strings from webscraping
df["heat"] = df["heat"].map(
    {
        "Heat 1 ": "Heat 1",
        "Heat 2 ": "Heat 2",
        "Heat 3 ": "Heat 3",
        "Heat 4 ": "Heat 4",
        "Heat 5 ": "Heat 5",
        "Heat 6 ": "Heat 6",
        "Heat 7 ": "Heat 7",
        "Heat 8 ": "Heat 8",
        "Heat 9 ": "Heat 9",
        "Heat 10 ": "Heat 10",
        "Heat 11 ": "Heat 11",
        "Heat 12 ": "Heat 12",
        "Heat 13 ": "Heat 13",
        "Heat 14 ": "Heat 14",
        "Heat 15 ": "Heat 15",
        "Heat 16 ": "Heat 16",
        "Heat 8 Watch replay": "Heat 8",
        "Heat 6 Watch replay": "Heat 6",
        "Heat 2 Watch replay": "Heat 2",
        "Heat 1 Watch replay": "Heat 1",
        "Heat 4 Watch replay": "Heat 4",
        "Heat 10 Watch replay": "Heat 10",
        "Heat 11 Watch replay": "Heat 11",
        "Heat 12 Watch replay": "Heat 12",
        "Heat 3 Watch replay": "Heat 3",
        "Heat 5 Watch replay": "Heat 5",
        "Heat 7 Watch replay": "Heat 7",
        "Heat 9 Watch replay": "Heat 9",
        "Heat 13 Watch replay": "Heat 13",
        "Heat 14 Watch replay": "Heat 14",
        "Heat 15 Watch replay": "Heat 15",
        "Heat 16 Watch replay": "Heat 16",
    }
)


## Focus on Pipeline data

**Reindex doesn't work as expected**

In [11]:
# Get only pipeline scores
# Uses 'pipe' list from earlier
df_pipe = df.loc[df["event"].isin(pipe)].reindex()
df_pipe


,year,event,round,heat,name,score
0,2008,Billabong Pipeline Masters,Round 2,Heat 1,Kamalei Alexander,11.90
1,2008,Billabong Pipeline Masters,Round 2,Heat 1,Mikael Picon,11.67
2,2008,Billabong Pipeline Masters,Round 2,Heat 2,Evan Valiere,14.57
3,2008,Billabong Pipeline Masters,Round 2,Heat 2,Mick Campbell,14.34
4,2008,Billabong Pipeline Masters,Round 2,Heat 3,Jamie O'Brien,15.00
...,...,...,...,...,...,...
105,2022,Billabong Pro Pipeline,Semifinals,Heat 1,Miguel Pupo,9.50
106,2022,Billabong Pro Pipeline,Semifinals,Heat 2,Seth Moniz,13.50
107,2022,Billabong Pro Pipeline,Semifinals,Heat 2,Caio Ibelli,6.33
108,2022,Billabong Pro Pipeline,Final,Heat 1,Kelly Slater,18.77


## Load event dates data

This data was scraped idependently from scores to avoid using selenium

In [12]:
# Pipeline heat dates scraped from /results endpoint instead of main
event_dates = pd.read_csv("../data/wsl/pipe_masters_heat_dates.csv", index_col=0)
event_dates.head()


,year,event,round,heat,date
0,2008,Billabong Pipeline Masters,Round 1,Heat 1,
1,2008,Billabong Pipeline Masters,Round 1,Heat 2,
2,2008,Billabong Pipeline Masters,Round 1,Heat 3,
3,2008,Billabong Pipeline Masters,Round 1,Heat 4,
4,2008,Billabong Pipeline Masters,Round 1,Heat 5,


### Missing data

Events before 2014 have no heat dates. Have contacted WSL to inquire about getting this data in a structured way.

Other option would be to investigate news articles manually but I'll do that if I have time after project is completed

In [13]:
event_dates["date"].value_counts()


                      330
 December 19, 2014     27
 February 1, 2022      24
 December 16, 2018     24
 December 17, 2017     24
 December 18, 2016     23
 December 17, 2020     22
 December 16, 2015     19
 January 29, 2022      16
 December 11, 2019     16
 December 19, 2016     16
 December 10, 2019     16
 December 12, 2018     15
 December 19, 2019     15
 December 18, 2017     15
 December 12, 2014     12
 December 13, 2018     12
 December 14, 2016     12
 December 9, 2020      12
 December 11, 2015     12
 December 20, 2020     12
 December 10, 2015     12
 December 13, 2014     12
 December 17, 2018     11
 December 11, 2017      9
 December 17, 2015      8
 February 5, 2022       7
 December 16, 2017      3
 December 14, 2020      1
Name: date, dtype: int64

In [14]:
# Sanity check
event_dates[event_dates["year"] == 2022]["round"].value_counts()


Round of 32          16
Opening Round        12
Round of 16           8
Elimination Round     4
Quarterfinals         4
Semifinals            2
Final                 1
Name: round, dtype: int64

### Round and heat names have to be mapped like score data

In [15]:
event_dates["round"].unique()


array(['Round 1', 'Round 2', 'Round 3', 'Round 4', 'Quarterfinal',
       'Semifinal', 'Final', 'Round 5', 'Quarterfinals', 'Semifinals',
       'Trials Round 1', 'Trials Quarterfinals', 'Trials Semifinals',
       'Trials Final', 'Seeding Round', 'Elimination Round',
       'Round of 32', 'Round of 16', 'Opening Round'], dtype=object)

In [16]:
df_pipe["round"].unique()


array(['Round 2', 'Round 1', 'Round 3', 'Round 4', 'Quarterfinals',
       'Semifinals', 'Final', 'Round 5', 'Trials Quarterfinals',
       'Trials Round 1', 'Trials Semifinals', 'Trials Final'],
      dtype=object)

In [17]:
event_dates["round"] = event_dates["round"].map(
    {
        "Seeding Round": "Round 1",
        "Round of 32": "Round 3",
        "Quarterfinal": "Quarterfinals",
        "Round of 16": "Round 4",
        "Elimination Round": "Round 2",
        "Opening Round": "Round 1",
        "Round 1": "Round 1",
        "Round 2": "Round 2",
        "Round 3": "Round 3",
        "Round 4": "Round 4",
        "Round 5": "Round 5",
        "Quarterfinals": "Quarterfinals",
        "Final": "Final",
        "Semifinal": "Semifinals",
        "Semifinals": "Semifinals",
        "Trials Quarterfinals": "Trials Quarterfinals",
        "Trials Round 1": "Trials Round 1",
        "Trials Semifinals": "Trials Semifinals",
        "Trials Final": "Trials Final",
    }
)


In [18]:
event_dates["heat"].unique()


array(['Heat 1', 'Heat 2', 'Heat 3', 'Heat 4', 'Heat 5', 'Heat 6',
       'Heat 7', 'Heat 8', 'Heat 9', 'Heat 10', 'Heat 11', 'Heat 12',
       'Heat 13', 'Heat 14', 'Heat 15', 'Heat 16'], dtype=object)

In [19]:
df_pipe["heat"].unique()


array(['Heat 1', 'Heat 2', 'Heat 3', 'Heat 4', 'Heat 5', 'Heat 6',
       'Heat 7', 'Heat 8', 'Heat 9', 'Heat 10', 'Heat 11', 'Heat 12',
       'Heat 13', 'Heat 14', 'Heat 15', 'Heat 16'], dtype=object)

In [20]:
df_pipe["heat"].unique() == event_dates["heat"].unique()


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [21]:
df_pipe["round"].unique() == event_dates["round"].unique()


array([False, False,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True])

### Convert dates to datetime

In [22]:
# Check unique values
event_dates["date"].unique()


array([' ', ' December 12, 2014', ' December 13, 2014',
       ' December 19, 2014', ' December 10, 2015', ' December 11, 2015',
       ' December 16, 2015', ' December 17, 2015', ' December 14, 2016',
       ' December 18, 2016', ' December 19, 2016', ' December 11, 2017',
       ' December 16, 2017', ' December 17, 2017', ' December 18, 2017',
       ' December 12, 2018', ' December 13, 2018', ' December 16, 2018',
       ' December 17, 2018', ' December 10, 2019', ' December 11, 2019',
       ' December 19, 2019', ' December 9, 2020', ' December 14, 2020',
       ' December 17, 2020', ' December 20, 2020', ' January 29, 2022',
       ' February 1, 2022', ' February 5, 2022'], dtype=object)

In [23]:
# Eliminate first character blank space
event_dates["date"] = event_dates["date"].str[1:]


Keep only rows with date data

In [24]:
event_dates = event_dates[event_dates["date"] != ""]


In [25]:
event_dates["date"] = pd.to_datetime(event_dates["date"])
event_dates["date"].unique()


array(['2014-12-12T00:00:00.000000000', '2014-12-13T00:00:00.000000000',
       '2014-12-19T00:00:00.000000000', '2015-12-10T00:00:00.000000000',
       '2015-12-11T00:00:00.000000000', '2015-12-16T00:00:00.000000000',
       '2015-12-17T00:00:00.000000000', '2016-12-14T00:00:00.000000000',
       '2016-12-18T00:00:00.000000000', '2016-12-19T00:00:00.000000000',
       '2017-12-11T00:00:00.000000000', '2017-12-16T00:00:00.000000000',
       '2017-12-17T00:00:00.000000000', '2017-12-18T00:00:00.000000000',
       '2018-12-12T00:00:00.000000000', '2018-12-13T00:00:00.000000000',
       '2018-12-16T00:00:00.000000000', '2018-12-17T00:00:00.000000000',
       '2019-12-10T00:00:00.000000000', '2019-12-11T00:00:00.000000000',
       '2019-12-19T00:00:00.000000000', '2020-12-09T00:00:00.000000000',
       '2020-12-14T00:00:00.000000000', '2020-12-17T00:00:00.000000000',
       '2020-12-20T00:00:00.000000000', '2022-01-29T00:00:00.000000000',
       '2022-02-01T00:00:00.000000000', '2022-02-05

Save to csv and assign offsets in Excel

In [26]:
event_dates.to_csv("../data/wsl/pipeline_dates.csv")


Read in excel with offsets. This could be accomplished with a .apply function

In [27]:
event_dates_offset = pd.read_csv("../data/wsl/pipeline_dates_offset.csv", index_col=0)
# Convert date column to datetime format
event_dates_offset["date"] = pd.to_datetime(event_dates_offset["date"])
event_dates_offset['year'] = event_dates_offset['year'].astype(str)
event_dates_offset.head()


,year,event,round,heat,date,hour
0,2014,Billabong Pipe Masters,Round 1,Heat 1,2014-12-12,6.0
1,2014,Billabong Pipe Masters,Round 1,Heat 2,2014-12-12,6.5
2,2014,Billabong Pipe Masters,Round 1,Heat 3,2014-12-12,7.0
3,2014,Billabong Pipe Masters,Round 1,Heat 4,2014-12-12,7.5
4,2014,Billabong Pipe Masters,Round 1,Heat 5,2014-12-12,8.0


### Assign estimated times to heats based on dates and 30 minute heat times

In [28]:
# With help from
# https://www.codegrepper.com/code-examples/python/how+to+add+30+minutes+in+datetime+column+in+pandas
event_dates_offset["est_datetime"] = event_dates_offset.apply(
    lambda x: x["date"] + pd.DateOffset(hours=x["hour"]),
    axis=1,
)


In [29]:
event_dates_offset.head()


,year,event,round,heat,date,hour,est_datetime
0,2014,Billabong Pipe Masters,Round 1,Heat 1,2014-12-12,6.0,2014-12-12 06:00:00
1,2014,Billabong Pipe Masters,Round 1,Heat 2,2014-12-12,6.5,2014-12-12 06:30:00
2,2014,Billabong Pipe Masters,Round 1,Heat 3,2014-12-12,7.0,2014-12-12 07:00:00
3,2014,Billabong Pipe Masters,Round 1,Heat 4,2014-12-12,7.5,2014-12-12 07:30:00
4,2014,Billabong Pipe Masters,Round 1,Heat 5,2014-12-12,8.0,2014-12-12 08:00:00


In [30]:
event_dates_offset.tail()

,year,event,round,heat,date,hour,est_datetime
42,2022,Billabong Pro Pipeline,Quarterfinals,Heat 3,2022-02-05,7.0,2022-02-05 07:00:00
43,2022,Billabong Pro Pipeline,Quarterfinals,Heat 4,2022-02-05,7.5,2022-02-05 07:30:00
44,2022,Billabong Pro Pipeline,Semifinals,Heat 1,2022-02-05,8.0,2022-02-05 08:00:00
45,2022,Billabong Pro Pipeline,Semifinals,Heat 2,2022-02-05,8.5,2022-02-05 08:30:00
46,2022,Billabong Pro Pipeline,Final,Heat 1,2022-02-05,9.0,2022-02-05 09:00:00


## Merge scores with times

In [31]:
scores_and_times = df_pipe.merge(
    event_dates_offset,
    left_on=["year", "event", "round", "heat"],
    right_on=["year", "event", "round", "heat"],
).drop(columns=["date", "hour"])
scores_and_times.head(30)


,year,event,round,heat,name,score,est_datetime
0,2014,Billabong Pipe Masters,Round 2,Heat 1,Kelly Slater,14.10,2014-12-13 06:00:00
1,2014,Billabong Pipe Masters,Round 2,Heat 1,Reef McIntosh,7.00,2014-12-13 06:00:00
2,2014,Billabong Pipe Masters,Round 2,Heat 2,Michel Bourez,8.33,2014-12-13 06:30:00
3,2014,Billabong Pipe Masters,Round 2,Heat 2,Makai McNamara,4.57,2014-12-13 06:30:00
4,2014,Billabong Pipe Masters,Round 2,Heat 3,Dusty Payne,4.30,2014-12-13 07:00:00
5,2014,Billabong Pipe Masters,Round 2,Heat 3,Jordy Smith,4.30,2014-12-13 07:00:00
6,2014,Billabong Pipe Masters,Round 2,Heat 4,Nat Young,8.50,2014-12-13 07:30:00
7,2014,Billabong Pipe Masters,Round 2,Heat 4,Mitch Coleborn,4.77,2014-12-13 07:30:00
8,2014,Billabong Pipe Masters,Round 2,Heat 5,Miguel Pupo,7.17,2014-12-13 08:00:00
9,2014,Billabong Pipe Masters,Round 2,Heat 5,Raoni Monteiro,4.13,2014-12-13 08:00:00


In [32]:
scores_cleaned_merged = pd.DataFrame(scores_and_times.groupby(by=['est_datetime']).mean())
scores_cleaned_merged

,score
est_datetime,
2014-12-12 06:00:00,6.440000
2014-12-12 06:30:00,7.266667
2014-12-12 07:00:00,7.223333
2014-12-12 07:30:00,15.180000
2014-12-12 08:00:00,4.786667
...,...
2022-02-05 07:00:00,17.300000
2022-02-05 07:30:00,6.700000
2022-02-05 08:00:00,9.630000


## Save scores and times to csv

In [33]:
scores_cleaned_merged.to_csv('../data/wsl/pipe_2014_2022_clean_dated.csv')

## Get buoy data, wind data; and merge with scores and times

In [34]:
noaa = pd.read_csv('../data/noaa/pipeline_buoy_full_shifted.csv', index_col=0, parse_dates=True)
noaa

,wave_height,dominant_period,avg_period,dominant_wave_direction_sin,dominant_wave_direction_cos,wave_height_1.0_h,dominant_period_1.0_h,avg_period_1.0_h,dominant_wave_direction_sin_1.0_h,dominant_wave_direction_cos_1.0_h,...,wave_height_6.0_h,dominant_period_6.0_h,avg_period_6.0_h,dominant_wave_direction_sin_6.0_h,dominant_wave_direction_cos_6.0_h,wave_height_9.0_h,dominant_period_9.0_h,avg_period_9.0_h,dominant_wave_direction_sin_9.0_h,dominant_wave_direction_cos_9.0_h
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01 00:00:00,1.48,11.11,7.91,-0.656059,0.754710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-01 00:30:00,1.53,11.76,8.05,-0.656059,0.754710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-01 01:30:00,1.57,10.53,7.89,-0.573576,0.819152,1.48,11.11,7.91,-0.656059,0.754710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-01 02:00:00,1.51,11.76,7.85,-0.573576,0.819152,1.53,11.76,8.05,-0.656059,0.754710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-01 02:30:00,1.43,8.33,7.54,-0.121869,0.992546,1.57,10.53,7.89,-0.573576,0.819152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-10 04:30:00,1.20,12.00,7.70,-0.292372,0.956305,1.20,9.00,7.00,0.000000,1.000000,...,1.2,13.0,7.9,-0.406737,0.913545,1.4,12.0,8.1,-0.469472,0.882948
2022-03-10 05:00:00,1.20,12.00,7.40,-0.515038,0.857167,1.20,11.00,7.10,-0.292372,0.956305,...,1.3,11.0,8.1,-0.358368,0.933580,1.5,13.0,8.5,-0.422618,0.906308
2022-03-10 05:30:00,1.30,9.00,7.50,0.224951,0.974370,1.20,12.00,7.70,-0.292372,0.956305,...,1.3,13.0,7.9,-0.515038,0.857167,1.5,10.0,8.1,0.034899,0.999391


In [35]:
wind = pd.read_csv('../data/wind/oahu_wind.csv', index_col=0, parse_dates=True)
wind

,wind_speed,gust_speed,wind_direction_sin,wind_direction_cos
date,,,,
2014-01-01 00:00:00,0.28,0.78,0.907777,-0.419452
2014-01-01 00:30:00,0.18,0.44,0.866025,-0.500000
2014-01-01 01:00:00,0.26,0.58,0.866025,-0.500000
2014-01-01 01:30:00,0.06,0.34,0.999701,-0.024432
2014-01-01 02:00:00,0.28,0.54,0.819152,0.573576
...,...,...,...,...
2022-03-10 05:30:00,0.30,0.30,0.615661,0.788011
2022-03-10 06:00:00,0.10,0.70,0.104528,0.994522
2022-03-10 06:30:00,0.20,0.60,0.994522,0.104528


In [36]:
data = noaa.merge(
    scores_cleaned_merged.merge(wind, how="left", left_index=True, right_index=True),
    left_index=True,
    right_index=True,
)
data.head()


,wave_height,dominant_period,avg_period,dominant_wave_direction_sin,dominant_wave_direction_cos,wave_height_1.0_h,dominant_period_1.0_h,avg_period_1.0_h,dominant_wave_direction_sin_1.0_h,dominant_wave_direction_cos_1.0_h,...,wave_height_9.0_h,dominant_period_9.0_h,avg_period_9.0_h,dominant_wave_direction_sin_9.0_h,dominant_wave_direction_cos_9.0_h,score,wind_speed,gust_speed,wind_direction_sin,wind_direction_cos
2014-12-12 06:00:00,2.56,11.76,9.82,-0.681998,0.731354,2.52,13.33,9.60,-0.669131,0.743145,...,2.94,13.33,10.56,-0.719340,0.694658,6.440000,0.60,1.05,0.410719,0.911762
2014-12-12 06:30:00,2.46,13.33,9.98,-0.681998,0.731354,2.40,13.33,9.99,-0.669131,0.743145,...,2.80,14.29,10.01,-0.731354,0.681998,7.266667,0.28,0.82,0.671721,-0.740805
2014-12-12 07:00:00,2.20,12.50,9.79,-0.681998,0.731354,2.56,11.76,9.82,-0.681998,0.731354,...,2.71,13.33,9.84,-0.681998,0.731354,7.223333,0.20,0.78,0.972776,-0.231748
2014-12-12 07:30:00,2.33,11.76,9.75,-0.681998,0.731354,2.46,13.33,9.98,-0.681998,0.731354,...,2.67,13.33,9.67,-0.694658,0.719340,15.180000,0.28,0.70,0.829038,0.559193
2014-12-12 08:00:00,2.48,11.76,10.13,-0.529919,0.848048,2.20,12.50,9.79,-0.681998,0.731354,...,2.65,14.29,9.92,-0.719340,0.694658,4.786667,0.52,0.90,-0.923210,0.384295


In [37]:
# Sanity check
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 348 entries, 2014-12-12 06:00:00 to 2022-02-05 09:00:00
Data columns (total 35 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   wave_height                        348 non-null    float64
 1   dominant_period                    348 non-null    float64
 2   avg_period                         348 non-null    float64
 3   dominant_wave_direction_sin        348 non-null    float64
 4   dominant_wave_direction_cos        348 non-null    float64
 5   wave_height_1.0_h                  348 non-null    float64
 6   dominant_period_1.0_h              348 non-null    float64
 7   avg_period_1.0_h                   348 non-null    float64
 8   dominant_wave_direction_sin_1.0_h  348 non-null    float64
 9   dominant_wave_direction_cos_1.0_h  348 non-null    float64
 10  wave_height_1.5_h                  348 non-null    float64
 11  dominant_period_1.5_h

## Save EDA and Modeling data

In [38]:
data.to_csv('../data/merged_data.csv')

## Linear Model fun

In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

In [40]:
X = data.drop(
    columns=[
        "score",
    ]
)
y = data['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1331)

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.fit_transform(X_test)

linreg = LinearRegression()
linreg.fit(X_train_sc, y_train)
print(linreg.score(X_test_sc, y_test))

0.026965553378852203


In [41]:
pd.DataFrame(linreg.coef_, X.columns).sort_values(by=0, ascending=False)

,0
avg_period_1.0_h,1.488327
wave_height_1.5_h,0.826082
dominant_wave_direction_cos,0.766997
dominant_wave_direction_sin_6.0_h,0.707394
avg_period_9.0_h,0.685031
wave_height,0.505124
wave_height_6.0_h,0.465903
wind_direction_cos,0.360936
dominant_wave_direction_cos_3.0_h,0.344531
dominant_period,0.208909


In [42]:
mean_squared_error(y_test, linreg.predict(X_test_sc), squared=False)

2.9537898032487377